### Data Exploration

First, the training data is imported and an overall description is provided.  This will give us a high level view of the data, including its span and variance.  The testing dataset is also imported at this stage, but will be held until the model is tested. 

In [5]:
import pandas as pd
import numpy as np

raw_train_set = pd.read_csv('CBC_3200.csv')
raw_test_set = pd.read_csv('CBC_800.csv')

raw_train_set.head()

,Seq#,ID#,Gender,M,R,F,FirstPurch,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt,Florence
0,1712,14148,0,218,2,2,4,0,0,0,0,0,0,0,0,0,0,1
1,3042,25193,1,217,6,1,6,0,0,0,0,0,0,0,0,0,0,0
2,204,1784,1,32,12,1,12,0,0,0,0,0,0,0,0,0,0,0
3,3628,30102,0,199,8,2,10,0,0,0,0,0,1,2,0,0,0,0
4,444,3989,1,203,16,2,24,1,0,1,0,0,0,0,1,0,0,0


The attributes: 'Seq#', "ID#', and 'Gender' are all nominal types as they only show distinction. 'Gender' is binarized with 0=Male and 1=Female. The remaining attributes are ratio types as a value of 0 has an important meaning.  For example, if a customer has a value of 0 for 'ArtBks', this means that they purchased zero books from the art book section-this could influence their probability of purchasing *The Art History of Florence*. If 'M' or 'F' is 0, that equates to the customer spending $0 at the bookstore (or no purchases).  While a value of 0 for 'M' does not exist in this dataset since a customer had to purchase something to be in the database, I would argue that this attribute is ratio. Lastly, 'R' is ratio as this signifies the number of months since a last purchase, a calue of 0 would equate to a purchase being the month the data was collected.

In [6]:
raw_train_set.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Seq#,3200.0,2003.824375,1150.290162,1.0,1015.75,2019.5,2996.25,4000.0
ID#,3200.0,16621.452187,9446.798182,25.0,8387.25,16703.0,24799.50,32977.0
Gender,3200.0,0.703750,0.456674,0.0,0.00,1.0,1.00,1.0
M,3200.0,208.002813,100.440485,15.0,128.00,208.0,284.00,475.0
R,3200.0,13.431875,8.085878,2.0,8.00,12.0,16.00,36.0
F,3200.0,3.780000,3.440065,1.0,1.00,2.0,6.00,12.0
FirstPurch,3200.0,26.277188,18.152209,2.0,12.00,20.0,36.00,99.0
ChildBks,3200.0,0.627188,0.994866,0.0,0.00,0.0,1.00,7.0
YouthBks,3200.0,0.306875,0.617209,0.0,0.00,0.0,0.00,5.0
CookBks,3200.0,0.724063,1.084914,0.0,0.00,0.0,1.00,7.0


In [8]:
print(raw_train_set.shape)

(3200, 18)


The training set has 3200 objects (customers), 17 attributes and 1 class(Florence).  It is also observed that the value span are relatively close, with the exception of 'M', which represents the totalized dollar amount the customer spent at the bookstore. This is important in determining if normalization should be performed.  'Seq#' and 'ID#' will be discussed later. The next step is to see if there is any missing data.

In [59]:
for i in raw_train_set:
    for k in raw_test_set[i].isna().items():
        if k[1]:
            print(True)


Since the above code segment didn't print out True, it is confirmed that there is no missing data in our training dataset. Next, the variables not needed will be removed from the dataset.  This includes 'Seq#' and 'ID#'.  These are not value added as are identification numbers assigned to the customer in the dataset, or the entire customer database the bookstore has. There is no information on how these numbers are assigned, but they are assumed to be sequential in nature.  Since the 4000 customers were randomly selected, there should be no correlation between 'Seq#' and 'ID#' and which books are purchased. 

In [63]:
train_set = raw_train_set.drop(['Seq#', 'ID#'], axis=1)
